<a href="https://colab.research.google.com/github/mtoce/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Assig1_LS_DS_121_Statistics_Probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
# Ask the question about naming the assignment. Should I delete the one that has an unchanged title?
# I move the assigment to the front for tab reasons

In [0]:
import pandas as pd
import numpy as np
from scipy import stats

**Goal 1: Load and clean the data (or determine the best method to drop observations when running tests)**

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-03-16 20:37:51--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.07s   

2020-03-16 20:37:51 (271 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
df = pd.read_csv('house-votes-84.data', names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Need numeric values, not strings :D
df = df.replace({'y': 1, 'n': 0, '?': np.NaN})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# Extra insight into our data which could be very useful to reaching goals 2-4
df.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Create new DFs for different parties
rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


**Goal 2: Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01**

In [0]:
# Insights into our data
dem.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Insights into our data
rep.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# t-test for dem vs reps on 'budget'
stats.ttest_ind(rep['budget'], dem['budget'], nan_policy='omit')

# With a p-value of 2e-7, it is clear that they are bipartisan

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
# check rep budget mean
rep['budget'].mean()

0.13414634146341464

In [0]:
# check dem budget mean
dem['budget'].mean()

# These means show that budget is clearly much more supported by dems than reps

0.8884615384615384

**Goal 3: Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01**

In [0]:
# t-test for reps vs. dems on 'right-to-sue'
stats.ttest_ind(rep['right-to-sue'], dem['right-to-sue'], nan_policy='omit')

# with a p-value of 1.2e-34, it's clear this issue is very bipartisan

Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)

In [0]:
# check rep budget mean
rep['right-to-sue'].mean()

0.8607594936708861

In [0]:
# check dem budget mean
dem['right-to-sue'].mean()

# These means show that reps support the right-to-sue over dems

0.2896825396825397

**Goal 4: Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)**

In [0]:
stats.ttest_ind(rep['immigration'], dem['immigration'], nan_policy='omit')

0.08330248490425066

In [0]:
# Wow ! That's actually close to what we need! A p-value of > .1
# Let's check the means just for some clarity

In [0]:
rep['immigration'].mean()

0.5575757575757576

In [0]:
dem['immigration'].mean()

0.4714828897338403

In [0]:
# Must just be coincidence, but the difference between means is .08 and p = .08

In [0]:
# from .describe above, looks like water project is very closely supported

stats.ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')


Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
# Nice ! p = .929 > .1 We've accomplished our 4th goal

In [0]:
rep['water-project'].mean()

0.5067567567567568

In [0]:
dem['water-project'].mean()

0.502092050209205

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

In [0]:
# Stretch goal 1: Refactor your code into functions so it's easy to rerun with
# arbitrary variables

In [0]:
# Loop over all columns in rep and dem DFs

df.columns

Index(['party', 'handicapped-infants', 'water-project', 'budget',
       'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
       'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration',
       'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free',
       'south-africa'],
      dtype='object')

In [0]:
def ttest(x, y):
  # print(stats.ttest_ind(x, y, nan_policy='omit'))
  t_val, p_val = stats.ttest_ind(x, y, nan_policy='omit')
  mean_diff = x.mean() - y.mean()       # x is first!
  return t_val, p_val, mean_diff

In [0]:
# Check to make sure it works as intended
ttest(rep['water-project'], dem['water-project'])

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)


(0.08896538137868286, 0.9291556823993485, 0.0046647065475518)

In [0]:
cols = list(df.columns)
cols

['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free',
 'south-africa']

In [0]:
# popping the party off of our original df
party_col = df.pop('party')
party_col

0      republican
1      republican
2        democrat
3        democrat
4        democrat
          ...    
430    republican
431      democrat
432    republican
433    republican
434    republican
Name: party, Length: 435, dtype: object

In [0]:
# Create an empty dict. The keys will be our column headers, the values
# will be the returned values from our function, t_val, p_val, and mean_diff
congressional_issues = {}

# Now loop over the column header list
for i in cols:
  # Normally you would .append here for a list, however for a dict you equate the key to the value
  congressional_issues[i] = ttest(rep[i], dem[i])
congressional_issues    # simply type the dict to see inside
                        # we can peer inside by using its keys shown below

{'aid-to-contras': (-18.052093200819733,
  2.82471841372357e-54,
  -0.6760310963648253),
 'anti-satellite-ban': (-12.526187929077842,
  8.521033017443867e-31,
  -0.5314600314600315),
 'budget': (-23.21277691701378, 2.0703402795404463e-77, -0.7543151969981238),
 'crime': (16.342085656197696, 9.952342705606092e-47, 0.6311719070981463),
 'duty-free': (-12.853146132542978,
  5.997697174347365e-32,
  -0.5477066094595975),
 'education': (20.500685724563073, 1.8834203990450192e-64, 0.7263894286824718),
 'el-salvador-aid': (21.13669261173219,
  5.600520111729011e-68,
  0.7358288770053476),
 'handicapped-infants': (-9.205264294809222,
  1.613440327937243e-18,
  -0.41677237491190977),
 'immigration': (1.7359117329695164, 0.08330248490425066, 0.08609286784191733),
 'mx-missile': (-16.437503268542994,
  5.03079265310811e-47,
  -0.6429130009775171),
 'physician-fee-freeze': (49.36708157301406,
  1.994262314074344e-177,
  0.9338247338247339),
 'religious-groups': (9.737575825219457,
  2.393672252059

In [0]:
congressional_issues['budget']

(-23.21277691701378, 2.0703402795404463e-77, -0.7543151969981238)

In [0]:
# Including the loop in a function that will do a ttest2 for any 2 DFs 
#   DISCLAIMER: DFs need to be the same shape to run test
def frame_ttest2(df1, df2):
  dictionary = {}
  for i in cols:
    dictionary[i] = ttest(df1[i], df2[i])
  return dictionary

frame_ttest2(rep, dem)

{'aid-to-contras': (-18.052093200819733,
  2.82471841372357e-54,
  -0.6760310963648253),
 'anti-satellite-ban': (-12.526187929077842,
  8.521033017443867e-31,
  -0.5314600314600315),
 'budget': (-23.21277691701378, 2.0703402795404463e-77, -0.7543151969981238),
 'crime': (16.342085656197696, 9.952342705606092e-47, 0.6311719070981463),
 'duty-free': (-12.853146132542978,
  5.997697174347365e-32,
  -0.5477066094595975),
 'education': (20.500685724563073, 1.8834203990450192e-64, 0.7263894286824718),
 'el-salvador-aid': (21.13669261173219,
  5.600520111729011e-68,
  0.7358288770053476),
 'handicapped-infants': (-9.205264294809222,
  1.613440327937243e-18,
  -0.41677237491190977),
 'immigration': (1.7359117329695164, 0.08330248490425066, 0.08609286784191733),
 'mx-missile': (-16.437503268542994,
  5.03079265310811e-47,
  -0.6429130009775171),
 'physician-fee-freeze': (49.36708157301406,
  1.994262314074344e-177,
  0.9338247338247339),
 'religious-groups': (9.737575825219457,
  2.393672252059

In [0]:
# Let's make sure our ttest works for another example. We'll use our original DF

frame_ttest2(df, rep)

{'aid-to-contras': (9.788808723718947,
  4.957020042243978e-21,
  0.4233242341522596),
 'anti-satellite-ban': (7.393093816126851,
  5.030722427335505e-13,
  0.3269552212545087),
 'budget': (11.065771123073784, 5.606429170374881e-26, 0.46255177174413253),
 'crime': (-9.863802115768447, 2.589969722941731e-21, -0.3880650242206306),
 'duty-free': (8.015250283138412, 6.412413068071732e-15, 0.33777483777483774),
 'education': (-10.38151321326716, 3.354231506184399e-23, -0.4477004152028106),
 'el-salvador-aid': (-11.06410974250642,
  5.842523567558973e-26,
  -0.4467532467532468),
 'handicapped-infants': (5.891208596021267,
  6.467122052729833e-09,
  0.2542015903718032),
 'immigration': (-1.1548576145820841,
  0.24861546277915458,
  -0.052902860379495986),
 'mx-missile': (9.181555136184896, 7.577133289155805e-19, 0.3860591386015115),
 'physician-fee-freeze': (-14.693048076924745,
  7.587344453307349e-42,
  -0.5704259576901087),
 'religious-groups': (-6.386795628256836,
  3.443895975403575e-10,